In [49]:
import json
import requests
import pandas as pd

# API KEY INPUT HERE
apikey = ""

def task1(stocks, apikey):
    url = "https://yfapi.net/v6/finance/quote"
    querystring = {"symbols": ",".join(stocks)}
    headers = {'x-api-key': apikey}

    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()

    if 'quoteResponse' in data and 'result' in data['quoteResponse']:
        for stock in data['quoteResponse']['result']:
            symbol = stock.get('symbol', 'N/A')
            company = stock.get('longName', 'N/A')
            price = stock.get('regularMarketPrice', 'N/A')

            if isinstance(price, (int, float)):
                price = f"${price:.2f}"
            else:
                price = "N/A"

            print(f"Stock Ticker: {symbol}, Company: {company}, Current Market Price: {price}")
    else:
        print("Invalid stock symbol")


def task2_1(symbol, apikey):
    url = f"https://yfapi.net/v11/finance/quoteSummary/{symbol}"
    querystring = {"modules": "summaryDetail,financialData"}
    headers = {'x-api-key': apikey}
    response = requests.get(url, headers=headers, params=querystring)

    try:
        data = response.json()
    except json.JSONDecodeError:
        print("Error: Unable to decode JSON response.")
        return

    if 'quoteSummary' in data and 'result' in data['quoteSummary'] and len(data['quoteSummary']['result']) > 0:
        stock_info = data['quoteSummary']['result'][0]
        summary_detail = stock_info.get("summaryDetail", {})
        financial_data = stock_info.get("financialData", {})
        extracted_data = {
            "Stock Ticker": symbol,
            "52 Week High": summary_detail.get("fiftyTwoWeekHigh", {}).get("raw", "N/A"),
            "52 Week Low": summary_detail.get("fiftyTwoWeekLow", {}).get("raw", "N/A"),
            "Return on Assets (ROA)": financial_data.get("returnOnAssets", {}).get("raw", "N/A")
        }
        df = pd.DataFrame([extracted_data])
        df["52 Week High"] = df["52 Week High"].apply(lambda x: f"${x:.2f}" if isinstance(x, (int, float)) else "N/A")
        df["52 Week Low"] = df["52 Week Low"].apply(lambda x: f"${x:.2f}" if isinstance(x, (int, float)) else "N/A")
        df["Return on Assets (ROA)"] = df["Return on Assets (ROA)"].apply(lambda x: f"{x:.2%}" if isinstance(x, (int, float)) else "N/A")
        print("\n2. Additional Stock Data (Converted to DataFrame):\n")
        print(df.to_string(index=False))

    else:
        print("Stock data error - API response does not contain expected data structure.")


def task2_2(apikey):
    url = "https://yfapi.net/v1/finance/trending/US"
    headers = {'x-api-key': apikey}
    response = requests.get(url, headers=headers)
    data = response.json()
    if "finance" in data and "result" in data["finance"] and len(data["finance"]["result"]) > 0:
        trending_symbols = [stock["symbol"] for stock in data["finance"]["result"][0]["quotes"]]
        query_url = "https://yfapi.net/v6/finance/quote"
        querystring = {"symbols": ",".join(trending_symbols)}
        response = requests.get(query_url, headers=headers, params=querystring)
        stock_data = response.json()

        if "quoteResponse" in stock_data and "result" in stock_data["quoteResponse"]:
            trending_stocks = []

            for stock in stock_data["quoteResponse"]["result"]:
                symbol = stock.get("symbol", "N/A")
                company = stock.get("longName", "N/A")
                price = stock.get("regularMarketPrice", "N/A")
                high_52 = stock.get("fiftyTwoWeekHigh", "N/A")
                low_52 = stock.get("fiftyTwoWeekLow", "N/A")

                price = f"${price:.2f}" if isinstance(price, (int, float)) else "N/A"
                high_52 = f"${high_52:.2f}" if isinstance(high_52, (int, float)) else "N/A"
                low_52 = f"${low_52:.2f}" if isinstance(low_52, (int, float)) else "N/A"

                trending_stocks.append({
                    "Stock Ticker": symbol,
                    "Company": company,
                    "Current Market Price": price,
                    "52 Week High": high_52,
                    "52 Week Low": low_52
                })

            df = pd.DataFrame(trending_stocks)

            print("\n3. Trending Stocks DataFrame:\n")
            print(df.to_string(index=False))

        else:
            print("Trending stocks error")
    else:
        print("Trending stocks error")


# Task 1
stock_symbols = input("1. Enter 2+ stock symbols (separate using commas): \n").strip().upper().split(",")
task1(stock_symbols, apikey)

# Task 2.1 (Fixed: Now Fetches All 3 Values)
symbol = input("\n2. Enter a stock symbol: ").strip().upper()
task2_1(symbol, apikey)

# Task 2.2 (Properly Displays Only DataFrame)
task2_2(apikey)


1. Enter 2+ stock symbols (separate using commas): 
AAPL,MSFT
Stock Ticker: AAPL, Company: Apple Inc., Current Market Price: $240.36
Stock Ticker: MSFT, Company: Microsoft Corporation, Current Market Price: $399.73

2. Enter a stock symbol: AAPL

2. Additional Stock Data (Converted to DataFrame):

Stock Ticker 52 Week High 52 Week Low Return on Assets (ROA)
        AAPL      $260.10     $164.08                 22.52%

3. Trending Stocks DataFrame:

Stock Ticker                              Company Current Market Price 52 Week High 52 Week Low
        NVDA                   NVIDIA Corporation              $131.28      $153.13      $75.61
        SNOW                       Snowflake Inc.              $166.19      $233.88     $107.13
         CRM                     Salesforce, Inc.              $307.33      $369.00     $212.00
        IONQ                           IonQ, Inc.               $29.93       $54.74       $6.22
        BYND                    Beyond Meat, Inc.                $3